In [1]:
%load_ext autoreload
%autoreload 2
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [1]:
from src.sparkclient import *

In [2]:
sc = SparkClient("XML_Import")

Ivy Default Cache set to: /Users/igor/.ivy2/cache
The jars for the packages stored in: /Users/igor/.ivy2/jars
:: loading settings :: url = jar:file:/Users/igor/.local/share/virtualenvs/data-studio-exercise-python-sql-x46TW6ms/lib/python3.8/site-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
com.databricks#spark-xml_2.12 added as a dependency
org.xerial#sqlite-jdbc added as a dependency
com.amazon.deequ#deequ added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-2b8d4603-63d9-4e3c-b8d0-df23d4e9ad3d;1.0
	confs: [default]
	found com.databricks#spark-xml_2.12;0.12.0 in central
	found commons-io#commons-io;2.8.0 in central
	found org.glassfish.jaxb#txw2;2.3.3 in central
	found org.apache.ws.xmlschema#xmlschema-core;2.2.5 in central
	found org.xerial#sqlite-jdbc;3.36.0.3 in central
	found com.amazon.deequ#deequ;1.2.2-spark-3.0 in central
	found org.scalanlp#breeze_2.12;0.13.2 in central
	found org.scalanlp#breeze-macros_2.1

In [3]:
sc.spark.stop()

# Parse XML

In [12]:
posts = Posts(sc, 'uncommitted/Posts.xml')
# posts.df.printSchema()

In [13]:
tags = Tags(sc, 'uncommitted/Tags.xml')
# tags.df.printSchema()

# Create posts-tags table

In [14]:
posts_tags = PostsTags(sc, posts.df, tags.df)
# posts_tags.df.printSchema()

# Save to Parquet

Saving data in Parquet format so that it is available for further analysis.
In production it is advisable to use partition for each load so that historical data is retained.

In [15]:
posts.save()
tags.save()
posts_tags.save()

# Constraints

In [15]:
# Generate suggested validation checks
# sc.suggest_constraints(posts.df)

In [16]:
# sc.suggest_constraints(tags.df)

In [17]:
# sc.suggest_constraints(posts_tags.df)

In [5]:
# Run Validation Checks for Posts
checkResult_df = posts.check()

Python Callback server started!


21/11/30 20:50:05 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [19]:
# Run Validation Checks for Tags
checkResult_df = checkResult_df.union(tags.check())

In [20]:
# Run Validation Checks for posts-tags
checkResult_df = checkResult_df.union(posts_tags.check())

In [21]:
# Save Check Results to Parquet
checkResult_df.write.mode('overwrite').parquet("uncommitted/check_results.parquet")

In [22]:
check_df = checkResult_df.toPandas()
failed_df = check_df[check_df['constraint_status'] != 'Success']
assert len(failed_df) == 0, "Data validation check has failed!"

# Save to SQLite

In [23]:
import src.db as sqldb

db = sqldb.Database('uncommitted/warehouse.db')

In [24]:
db.to_sql(checkResult_df.toPandas(), 'check_results')

In [25]:
sqldb.Posts(db).create().insert_df(posts.df)

In [26]:
sqldb.Tags(db).create().insert_df(tags.df)

In [27]:
sqldb.PostsTags(db).create().insert_df(posts_tags.df)